In [1]:
import math
#Tensorflow library. Used to implement machine learning models
import tensorflow as tf
#Numpy contains helpful functions for efficient mathematical calculations
import numpy as np
#Image library for image manipulation
from PIL import Image

/home/sidjarvis10/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
class RBM(object):
    
    def __init__(self, input_size, output_size):
        #Defining the hyperparameters
        self._input_size = input_size #Size of input
        self._output_size = output_size #Size of output
        self.epochs = 5 #Amount of training iterations
        self.learning_rate = 1.0 #The step used in gradient descent
        self.batchsize = 100 #The size of how much data will be used for training per sub iteration
        
        #Initializing weights and biases as matrices full of zeroes
        self.w = np.zeros([input_size, output_size], np.float32) #Creates and initializes the weights with 0
        self.hb = np.zeros([output_size], np.float32) #Creates and initializes the hidden biases with 0
        self.vb = np.zeros([input_size], np.float32) #Creates and initializes the visible biases with 0


    #Fits the result from the weighted visible layer plus the bias into a sigmoid curve
    def prob_h_given_v(self, visible, w, hb):
        #Sigmoid 
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)

    #Fits the result from the weighted hidden layer plus the bias into a sigmoid curve
    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)
    
    #Generate the sample probability
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))

    #Training method for the model
    def train(self, X):
        #Create the placeholders for our parameters
        _w = tf.placeholder("float", [self._input_size, self._output_size])
        _hb = tf.placeholder("float", [self._output_size])
        _vb = tf.placeholder("float", [self._input_size])
        
        prv_w = np.zeros([self._input_size, self._output_size], np.float32) #Creates and initializes the weights with 0
        prv_hb = np.zeros([self._output_size], np.float32) #Creates and initializes the hidden biases with 0
        prv_vb = np.zeros([self._input_size], np.float32) #Creates and initializes the visible biases with 0

        
        cur_w = np.zeros([self._input_size, self._output_size], np.float32)
        cur_hb = np.zeros([self._output_size], np.float32)
        cur_vb = np.zeros([self._input_size], np.float32)
        v0 = tf.placeholder("float", [None, self._input_size])
        
        #Initialize with sample probabilities
        h0 = self.sample_prob(self.prob_h_given_v(v0, _w, _hb))
        v1 = self.sample_prob(self.prob_v_given_h(h0, _w, _vb))
        h1 = self.prob_h_given_v(v1, _w, _hb)
        
        #Create the Gradients
        positive_grad = tf.matmul(tf.transpose(v0), h0)
        negative_grad = tf.matmul(tf.transpose(v1), h1)
        
        #Update learning rates for the layers
        update_w = _w + self.learning_rate *(positive_grad - negative_grad) / tf.to_float(tf.shape(v0)[0])
        update_vb = _vb +  self.learning_rate * tf.reduce_mean(v0 - v1, 0)
        update_hb = _hb +  self.learning_rate * tf.reduce_mean(h0 - h1, 0)
        
        #Find the error rate
        err = tf.reduce_mean(tf.square(v0 - v1))
        
        #Training loop
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            #For each epoch
            for epoch in range(self.epochs):
                #For each step/batch
                for start, end in zip(range(0, len(X), self.batchsize),range(self.batchsize,len(X), self.batchsize)):
                    batch = X[start:end]
                    #Update the rates
                    cur_w = sess.run(update_w, feed_dict={v0: batch, _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    cur_hb = sess.run(update_hb, feed_dict={v0: batch, _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    cur_vb = sess.run(update_vb, feed_dict={v0: batch, _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    prv_w = cur_w
                    prv_hb = cur_hb
                    prv_vb = cur_vb
                error=sess.run(err, feed_dict={v0: X, _w: cur_w, _vb: cur_vb, _hb: cur_hb})
                print ('Epoch: %d' % epoch,'reconstruction error: %f' % error)
            self.w = prv_w
            self.hb = prv_hb
            self.vb = prv_vb

    #Create expected output for our DBN
    def rbm_outpt(self, X):
        input_X = tf.constant(X)
        _w = tf.constant(self.w)
        _hb = tf.constant(self.hb)
        out = tf.nn.sigmoid(tf.matmul(input_X, _w) + _hb)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            return sess.run(out)

In [5]:
from tensorflow.examples.tutorials.mnist import input_data

#Loading in the mnist data
mnist = input_data.read_data_sets("/home/sidjarvis10/anaconda3/lib/python3.6/site-packages/tensorflow/examples/tutorials/mnist", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images,\
    mnist.test.labels

Extracting /home/sidjarvis10/anaconda3/lib/python3.6/site-packages/tensorflow/examples/tutorials/mnist/train-images-idx3-ubyte.gz
Extracting /home/sidjarvis10/anaconda3/lib/python3.6/site-packages/tensorflow/examples/tutorials/mnist/train-labels-idx1-ubyte.gz
Extracting /home/sidjarvis10/anaconda3/lib/python3.6/site-packages/tensorflow/examples/tutorials/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/sidjarvis10/anaconda3/lib/python3.6/site-packages/tensorflow/examples/tutorials/mnist/t10k-labels-idx1-ubyte.gz


In [9]:
RBM_hidden_sizes = [500, 200 , 50 ] #create 2 layers of RBM with size 400 and 100

#Since we are training, set input as training data
inpX = trX

#Create list to hold our RBMs
rbm_list = []

#Size of inputs is the number of inputs in the training set
input_size = inpX.shape[1]

#For each RBM we want to generate
for i, size in enumerate(RBM_hidden_sizes):
    print ('RBM: ',i,' ',input_size,'->', size)
    rbm_list.append(RBM(input_size, size))
    input_size = size

RBM:  0   784 -> 500
RBM:  1   500 -> 200
RBM:  2   200 -> 50


In [11]:
#For each RBM in our list
for rbm in rbm_list:
    print ('New RBM:')
    #Train a new one
    rbm.train(inpX) 
    #Return the output layer
    inpX = rbm.rbm_outpt(inpX)

New RBM:
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0 reconstruction error: 0.061014
Epoch: 1 reconstruction error: 0.052954
Epoch: 2 reconstruction error: 0.049889
Epoch: 3 reconstruction error: 0.047555
Epoch: 4 reconstruction error: 0.046133
New RBM:
Epoch: 0 reconstruction error: 0.033658
Epoch: 1 reconstruction error: 0.029916
Epoch: 2 reconstruction error: 0.028547
Epoch: 3 reconstruction error: 0.027253
Epoch: 4 reconstruction error: 0.026510
New RBM:
Epoch: 0 reconstruction error: 0.057651
Epoch: 1 reconstruction error: 0.054375
Epoch: 2 reconstruction error: 0.053073
Epoch: 3 reconstruction error: 0.051773
Epoch: 4 reconstruction error: 0.051451


In [50]:
import numpy as np
import math
import tensorflow as tf


class NN(object):
    
    def __init__(self, sizes, X, Y, X1, Y1):
        #Initialize hyperparameters
        self._sizes = sizes
        self._X = X
        self._Y = Y
        self._X1= X1
        self._Y1= Y1
        self.w_list = []
        self.b_list = []
        self.w1_list = []
        self.b1_list = []
        self._learning_rate =  1.0
        self._momentum = 0.0
        self._epoches = 20
        self._batchsize = 100
        input_size = X.shape[1]
        
        #initialization loop
        for size in self._sizes + [Y.shape[1]]:
            #Define upper limit for the uniform distribution range
            max_range = 4 * math.sqrt(6. / (input_size + size))
            
            #Initialize weights through a random uniform distribution
            self.w_list.append(
                np.random.uniform( -max_range, max_range, [input_size, size]).astype(np.float32))
            
            #Initialize bias as zeroes
            self.b_list.append(np.zeros([size], np.float32))
            input_size = size
      
    #load data from rbm
    def load_from_rbms(self, dbn_sizes,rbm_list):
        #Check if expected sizes are correct
        assert len(dbn_sizes) == len(self._sizes)
        
        for i in range(len(self._sizes)):
            #Check if for each RBN the expected sizes are correct
            assert dbn_sizes[i] == self._sizes[i]
        
        #If everything is correct, bring over the weights and biases
        for i in range(len(self._sizes)):
            self.w_list[i] = rbm_list[i].w
            self.b_list[i] = rbm_list[i].hb

    #Training method
    def train(self):
        #Create placeholders for input, weights, biases, output
        _a = [None] * (len(self._sizes) + 2)
        _w = [None] * (len(self._sizes) + 1)
        _b = [None] * (len(self._sizes) + 1)
        _a[0] = tf.placeholder("float", [None, self._X.shape[1]])
        y = tf.placeholder("float", [None, self._Y.shape[1]])
        
        #Define variables and activation functoin
        for i in range(len(self._sizes) + 1):
            _w[i] = tf.Variable(self.w_list[i])
            _b[i] = tf.Variable(self.b_list[i])
        for i in range(1, len(self._sizes) + 2):
            _a[i] = tf.nn.sigmoid(tf.matmul(_a[i - 1], _w[i - 1]) + _b[i - 1])
        
        #Define the cost function
        cost = tf.reduce_mean(tf.square(_a[-1] - y))
        
        #Define the training operation (Momentum Optimizer minimizing the Cost function)
        train_op = tf.train.MomentumOptimizer(
            self._learning_rate, self._momentum).minimize(cost)
        
        #Prediction operation
        predict_op = tf.argmax(_a[-1], 1)
        
        #Training Loop
        with tf.Session() as sess:
            #Initialize Variables
            sess.run(tf.global_variables_initializer())
            
            #For each epoch
            for i in range(self._epoches):
                
                #For each step
                for start, end in zip(
                    range(0, len(self._X), self._batchsize), range(self._batchsize, len(self._X), self._batchsize)):
                    
                    #Run the training operation on the input data
                    sess.run(train_op, feed_dict={
                        _a[0]: self._X[start:end], y: self._Y[start:end]})
                
                for j in range(len(self._sizes) + 1):
                    #Retrieve weights and biases
                    self.w_list[j] = sess.run(_w[j])
                    self.b_list[j] = sess.run(_b[j])
                
                print ("Accuracy rating for epoch " + str(i) + ": " + str(np.mean(np.argmax(self._Y, axis=1) ==
                              sess.run(predict_op, feed_dict={_a[0]: self._X, y: self._Y}))))
                
            self.w1_list = self.w_list
            self.b1_list = self.b_list
            _aa = [None] * (len(self._sizes) + 2)
            _ww = [None] * (len(self._sizes) + 1)
            _bb = [None] * (len(self._sizes) + 1)
            _aa[0] = tf.placeholder("float", [None, self._X1.shape[1]])
            yy = tf.placeholder("float", [None, self._Y1.shape[1]])
            for i in range(len(self._sizes) + 1):
                _ww[i] = tf.Variable(self.w1_list[i])
                _bb[i] = tf.Variable(self.b1_list[i])
            for i in range(1, len(self._sizes) + 2):
                _aa[i] = tf.nn.sigmoid(tf.matmul(_aa[i - 1], _ww[i - 1]) + _bb[i - 1])
                
            predict_test_op = tf.argmax(_aa[-1], 1)
        with tf.Session() as sess:
            #Initialize Variables
            sess.run(tf.global_variables_initializer())
            print ("Test rating for dataset " + ": " + str(np.mean(np.argmax(self._Y1, axis=1) ==
                              sess.run(predict_test_op, feed_dict={_aa[0]: self._X1, yy: self._Y1}))))
                            

In [51]:
nNet = NN(RBM_hidden_sizes, trX, trY, teX, teY)
nNet.load_from_rbms(RBM_hidden_sizes,rbm_list)
nNet.train()

Accuracy rating for epoch 0: 0.5067818181818182
Accuracy rating for epoch 1: 0.7395818181818182
Accuracy rating for epoch 2: 0.7884909090909091
Accuracy rating for epoch 3: 0.8065272727272728
Accuracy rating for epoch 4: 0.8783272727272727
Accuracy rating for epoch 5: 0.8951636363636364
Accuracy rating for epoch 6: 0.9026181818181818
Accuracy rating for epoch 7: 0.9088545454545455
Accuracy rating for epoch 8: 0.9134545454545454
Accuracy rating for epoch 9: 0.9174363636363636
Accuracy rating for epoch 10: 0.9209636363636363
Accuracy rating for epoch 11: 0.9236363636363636
Accuracy rating for epoch 12: 0.9256545454545455
Accuracy rating for epoch 13: 0.9277454545454545
Accuracy rating for epoch 14: 0.9294727272727272
Accuracy rating for epoch 15: 0.9309272727272727
Accuracy rating for epoch 16: 0.9322181818181818
Accuracy rating for epoch 17: 0.9338181818181818
Accuracy rating for epoch 18: 0.9351272727272727
Accuracy rating for epoch 19: 0.9366
Test rating for dataset : 0.9391
